In [1]:
import tvm
from tvm import relax
from tvm.script.parser import relax as R
import numpy as np

# Define an example IRModule
@tvm.script.ir_module
class InputModule:
    @R.function
    def main(
        x: R.Tensor((16, 16), "float32"), y: R.Tensor((16, 16), "float32")
    ) -> R.Tensor((16, 16), "float32"):
        with R.dataflow():
            z1 = R.multiply(x, y)
            z2 = R.add(z1, x)
            z3 = R.add(z1, z2)
            z4 = R.multiply(z3, z2)
            z5 = R.add(z4, z1)
            R.output(z5)
        return z5


mod = InputModule

/home/qzylalala/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
from tvm.relax.dpl import is_op, wildcard
patterns = [
        ("tensorrt.multiply", is_op("relax.multiply")(wildcard(), wildcard())),
        ("tensorrt.add", is_op("relax.add")(wildcard(), wildcard())),
]

In [3]:
from tvm import relax
mod1 = relax.transform.FuseOpsByPattern(patterns)(mod)
mod1.show()

In [4]:
mod2 = relax.transform.MergeCompositeFunctions()(mod1)
mod2.show()

In [6]:
mod3 = relax.transform.RunCodegen()(mod2)
mod3.show()
# Produced runtime module will be attached in the IRModule attribute.
print(f"TensorRT runtime module: {mod3.attrs['external_mods']}")

[16:50:30] /home/qzylalala/work_space/tvm/include/tvm/script/printer/ir_docsifier_functor.h:72: Warning: ObjectFunctor calls un-registered function on type: runtime.Module (token: ir). ObjectType: runtime.Module. Object: runtime.Module(0x501e968)


TVMError: Traceback (most recent call last):
  20: TVMFuncCall
  19: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > (tvm::runtime::ObjectRef const&, tvm::runtime::Optional<tvm::PrinterConfig> const&)>::AssignTypedLambda<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > (*)(tvm::runtime::ObjectRef const&, tvm::runtime::Optional<tvm::PrinterConfig> const&)>(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > (*)(tvm::runtime::ObjectRef const&, tvm::runtime::Optional<tvm::PrinterConfig> const&), std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  18: tvm::TVMScriptPrinter::Script[abi:cxx11](tvm::runtime::Obj